This code is for the XGBoost. 
XGBoost can regarded as a general version of gradient boost. 
In XGboost, the current model wnats to maximize the loss function L(y_i, F_(t-1)+output)+ 0.5*regular_coef*output.
the second term is regularization term.
For details, please refer to 
https://www.youtube.com/watch?v=OtD8wVaFm6E


In [1]:
def XGBoost (data , maxlayer, max_tree_number,discrete,learn_rate,regular ):
    n=len(data)
    current_data = data.copy()
    current_tree_series = []
    current_tree_series.append( sum(current_data[y])/(len(data)+regular) )
    current_predict = [ sum(current_data[y])/(len(data)+regular)  for i in range(0,n)]
    current_data['pre_predict'] = current_predict
    while (len(current_tree_series)<=max_tree_number):
        # you a whole data set, whose y value is residual. use this to create a tree. : create_tree(current_data, maxlayer, data_features_original)
# thorough this step you get a new tree
        data_features = data_features_original.copy()
        new_tree=createTree(current_data, 1, data_features,discrete,regular) # a tree always start from first layer
        print(new_tree)
        current_tree_series.append ( new_tree )
       
       # produce new prediction 
        new_predict = [ predict ( current_data.loc[i],new_tree ) for  i in range(0,n) ]
        # add it to current predict with a learning rate
        current_predict =list(map(lambda x,y:x+y, current_predict , [j * learn_rate for j in new_predict] ))
        current_data['pre_predict']=current_predict
     
        
        # use the residual as the y for the next tree.
        
        #print(sum(list(map(lambda x: abs(x), current_data[y]-current_data['pre_predict']))))
    return current_tree_series


In [9]:
maxlayer = 3
max_tree_number = 15
xixixi = XGBoost (data , maxlayer, max_tree_number ,discrete,0.25,0.1)

{'spInd': 'housing', 'spVal': 0.5, 'left': {'spInd': 'foreigner', 'spVal': 0.5, 'left': -1019.8217760641518, 'right': -101.5829169581571}, 'right': {'spInd': 'age', 'spVal': 35.714285714285715, 'left': -5.2983907082699355, 'right': 79.56377684698228}}
{'spInd': 'housing', 'spVal': 0.5, 'left': {'spInd': 'foreigner', 'spVal': 0.5, 'left': -767.6680402241146, 'right': -76.20211760059304}, 'right': {'spInd': 'age', 'spVal': 35.714285714285715, 'left': -3.974085372335797, 'right': 59.67823631472943}}
{'spInd': 'housing', 'spVal': 0.5, 'left': {'spInd': 'foreigner', 'spVal': 0.5, 'left': -577.8600083005699, 'right': -57.16278780620598}, 'right': {'spInd': 'age', 'spVal': 35.714285714285715, 'left': -2.980783301231625, 'right': 44.76273036266444}}
{'spInd': 'housing', 'spVal': 0.5, 'left': {'spInd': 'foreigner', 'spVal': 0.5, 'left': -434.98253372075874, 'right': -42.88049220500804}, 'right': {'spInd': 'age', 'spVal': 35.714285714285715, 'left': -2.2357519420075924, 'right': 33.5750878922342

In [8]:
# this function choose the best feature given current data set. 
n_seq=8
def BestFeature(Dataset, unvisited_feature,discrete,regular):
    # want to return the best feature and the resulting data split
    # but before this , you can already calculate the the loss of the current data
    
    threshold=[]
    loss=[]
    #print(unvisited_feature)
    for feature in unvisited_feature:
        if discrete[feature]=='yes': # this is discrete 
            # feature is name
            # feature level is number?
            # next, for each level in the feature_level, get
            loss.append(SplitDataSet(Dataset,feature, 0.5,regular)[0])
            threshold.append(0.5)
        else: # the feature is continuous 
            # we first generate a threshold sequence 
            thres_seq=[   min(Dataset[feature])+i*(max(Dataset[feature])-min(Dataset[feature]))/(n_seq-1) 
                      for i in range(1,n_seq-1)   ]
            loss_seq= [ SplitDataSet(Dataset,feature, x, regular )[0] for x in thres_seq]
            loss.append(min(loss_seq))
            threshold.append(thres_seq[loss_seq.index(min(loss_seq))])
        
    # we complete the loop with loss  series
    # we now return the best feature and the corresponding threshold 
    # for discrete, the corresponding threshold is always -1 
    best_feature=unvisited_feature[loss.index(min(loss))]
    best_threshold=threshold[loss.index(min(loss))]
    # we also return the splitted dat
    return best_feature,best_threshold
    # best_feature is a string. best_threshold is a value. 

# This function split the data into two parts given the feature and the threshold (value)
def SplitDataSet(Dataset,feature, value,regular):
# we only consider the binary classification case
# given the dataset, feature and value, this function returns splitted data and the resulting loss!
    Data_left=Dataset[Dataset[feature]<value]
    Data_right=Dataset[Dataset[feature]>=value]
# we want to calculate the loss of the data_left, data_right
    Total_loss=loss_cal(Data_left[y],Data_left['pre_predict'],regular
                      )+loss_cal(Data_right[y],Data_right['pre_predict'],regular)
    return Total_loss,Data_left, Data_right 

import math
from statistics import mean 
def loss_cal(y_data,pre_predict,regular):
    residual=y_data-pre_predict
    return(sum([pow(i,2)/(len(y_data)+regular) for i in residual]))


def createTree(Dataset, layer,features,discrete,regular):
    # we first need to check whether the layer has already reached the max
    if (layer==maxlayer  ):
        # you have already reached the deepest. 
        Tree=sum(Dataset[y]-Dataset['pre_predict'])/(len(Dataset)+regular)
    else:
        best_feature,best_threshold=BestFeature(Dataset, features,discrete,regular)
        Tree = {}  # means that you want to generate a new son tree
        # the new son tree should contain the following features:
        Tree['spInd'] = best_feature # the feature you use to split the data
        featuresNext=features
        featuresNext.remove(best_feature)  # this feature would no longer be used in the next trees
        Tree['spVal'] = best_threshold # the threshold value 
        data_left, data_right = SplitDataSet(Dataset,best_feature, best_threshold,regular)[1],SplitDataSet(
            Dataset,best_feature, best_threshold,regular)[2]
        Tree['left'] = createTree(data_left,layer+1,featuresNext,discrete,regular)
        Tree['right'] = createTree(data_right,layer+1,featuresNext,discrete,regular)
    return Tree

# the following is for prediction.
def predict(sample, tree):
    if (sample[tree['spInd']]<tree['spVal']):
        pred = predict(sample,tree['left']) if type(tree['left']).__name__=='dict' else tree['left']
    else:
        pred = predict(sample,tree['right']) if type(tree['right']).__name__=='dict' else  tree['right']
    return (pred)

In [3]:
import numpy as np
import pandas as pd 
data= pd.read_excel('C:/Users/shufe/Dropbox/ds new/pwchkt.xlsx')
data.isnull().any(axis=1) # 判定每行会不会有空值的记录。（某条记录但凡在一个变量上有空值，就认为此记录
data=data.dropna(how='any')
data=data[['age','creditamount','guarantors','housing','foreigner','creditrating']]
data.loc[(data.creditrating == 2),'creditrating']=0
data_features_original = list(data.columns) 
y='creditamount'
data_features_original.remove(y)
# we also need to create a dictionary indicating whether the feature is discrete or not
discrete={'age':'no','creditamount':'no','guarantors':'yes','housing':'yes','foreigner':'yes','creditrating':'yes'}
data_features=data_features_original.copy()
